#### Setup

In [ ]:
import polars as pl
from analysis.visualization.characterisation.clustering import cluster_timeseries_usage, usage_probabilities
from analysis.visualization.characterisation.notebooks.notebook_config import (
    dl,
    FEATURES,
    N_CLUSTERS,
    DATASET_START,
    DATASET_END,
    TIME_SERIES_MODE,
    WINDOW_MONTHS,
)

usage = cluster_timeseries_usage(
    loader=dl,
    k=N_CLUSTERS,
    features=FEATURES,
    start=DATASET_START,
    end=DATASET_END,
    mode=TIME_SERIES_MODE,
    window_months=WINDOW_MONTHS
)

usage_probs = usage_probabilities(usage).sort(["station", "probability"], descending=True)


## 8. Impact of Weather on Station Usage Patterns

### Event based effects

In [ ]:
rainy = pl.col("precipitation") >= 2.0
hot   = pl.col("temperature_2m") >= 23
cold  = pl.col("temperature_2m") <= 5
windy = pl.col("wind_speed_10m") > 6

wd = dl.get_weather(sample_rate="1d")

rain_intervals = wd.get_intervals(rainy)
hot_intervals = wd.get_intervals(hot)
cold_intervals = wd.get_intervals(cold)
windy_intervals = wd.get_intervals(windy)

In [ ]:
from analysis.visualization.characterisation.weather import compute_weather_deltas
from analysis.visualization.characterisation.helpers import label_deltas_with_usage


delta_rainy = compute_weather_deltas(loader=dl, event_intervals=rain_intervals, weekday=None, hours=None)
delta_hot = compute_weather_deltas(loader=dl, event_intervals=hot_intervals, weekday=None, hours=None)
delta_cold = compute_weather_deltas(loader=dl, event_intervals=cold_intervals, weekday=None, hours=None)
delta_windy = compute_weather_deltas(loader=dl, event_intervals=windy_intervals, weekday=None, hours=None)

delta_rainy_labeled = label_deltas_with_usage(delta_df=delta_rainy, usage_probs=usage_probs)
delta_hot_labeled = label_deltas_with_usage(delta_df=delta_hot, usage_probs=usage_probs)
delta_cold_labeled = label_deltas_with_usage(delta_df=delta_cold, usage_probs=usage_probs)
delta_windy_labeled = label_deltas_with_usage(delta_df=delta_windy, usage_probs=usage_probs)

In [ ]:
DELTA_REL = "delta_relative"  
DELTA_ABS = "delta_absolute"  

impact_rain = (
    delta_rainy_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_hot = (
    delta_hot_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_cold = (
    delta_cold_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_windy = (
    delta_windy_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

In [ ]:
impact_rain

In [ ]:
impact_hot

In [ ]:
impact_cold

In [ ]:
impact_windy

### Continuous responses

In [ ]:
from analysis.visualization.characterisation.plotting import plot_weather_response
from analysis.visualization.characterisation.weather import weather_response_by_usage, weather_response_df
from analysis.visualization.characterisation.helpers import dominant_usage_per_station


df = weather_response_df(loader=dl)

df = df.join(
    dominant_usage_per_station(usage_probs),
    on="station",
    how="left"
)

In [ ]:
temp_resp = weather_response_by_usage(
    df,
    var="temperature_2m",
    bin_width=0.3,
    baseline_cond=pl.col("temperature_2m") <= 10,
    min_var=10,
    max_var=29,
)

plot_weather_response(
    temp_resp,
    xlabel="Temperature (°C)",
    title="Temperature response by usage type",
)

In [ ]:
wind_resp = weather_response_by_usage(
    df,
    var="wind_speed_10m",
    bin_width=0.3,
    baseline_cond=pl.col("wind_speed_10m") <= 2,
    min_var=0.1,
    max_var=10,
)

plot_weather_response(
    wind_resp,
    xlabel="Wind speed (m/s)",
    title="Wind response by usage type",
)

In [ ]:
rain_resp = weather_response_by_usage(
    df,
    var="precipitation",
    bin_width=0.1,
    baseline_cond=pl.col("precipitation") == 0,
    min_var=0,
    max_var=2.5,
)

plot_weather_response(
    rain_resp,
    xlabel="Precipitation (mm)",
    title="Rain response by usage type",
)